In [1]:
!pip install feature-engine --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 kB 3.8 MB/s eta 0:00:00


In [8]:
import pandas as pd
import gdown
import json

## **Carga de los datasets**

Está compuesto por 3 partes:
* *pneumonia-gpi-icd10-mimic3.parquet.zip*: es un dataset anonimizado en formato tabular comprimido en formato parquet, que cuenta con 1683 filas que reportan ingresos de pacientes a un hospital. Las columnas poseen: el identificador del paciente, el identificador de admision, la etnia, el género, la variante de neumonía que podría tener el paciente (en código ICD9 - International Classification of Diseases), una serie de descriptores de medicamentos (drogas) en código NDC (National Drug Code), y una variable (target) que indica si el paciente tuvo o tiene chances de desarrollar neumonía.
* *ndc2gpi.json*: es un archivo en formato JSON que mapea un código NDC (National Drug Code) de descripción de un medicamento (droga) a su descripción y codigo GPI (Generic Product Identifier).
* *pneumonia-notes-mimic3.parquet.zip*: es un dataset anonimizado en formato texto semi-estructurado, que complementa al anterior y posee 555.338 notas del médico respecto a un  paciente dado en una determinada admisión al hospital. Cada fila incluye información del ID de admisión y del ID de paciente, la fecha, y un texto libro con las notas.

In [3]:
TABULAR_PNEUMONIA_URL = "https://drive.google.com/uc?export=download&id=1zrnaIUmfn7NBgFaLvU66xfVJmKTvgBjX"
# https://drive.google.com/file/d/1zrnaIUmfn7NBgFaLvU66xfVJmKTvgBjX/view?usp=sharing
TABULAR_PNEUMONIA_DATAPATH = 'pneumonia.zip'

gdown.download(TABULAR_PNEUMONIA_URL, TABULAR_PNEUMONIA_DATAPATH, quiet=True)

'pneumonia.zip'

In [4]:
NOTES_PNEUMONIA_URL = "https://drive.google.com/uc?export=download&id=1a_63mmcVXAJYBS5lB4PlrrFB4WmUfCxM"
# https://drive.google.com/file/d/1a_63mmcVXAJYBS5lB4PlrrFB4WmUfCxM/view?usp=sharing
NOTES_PNEUMONIA_DATAPATH = 'notes.zip'

gdown.download(NOTES_PNEUMONIA_URL, NOTES_PNEUMONIA_DATAPATH, quiet=True)

'notes.zip'

In [5]:
!unzip $TABULAR_PNEUMONIA_DATAPATH
!unzip $NOTES_PNEUMONIA_DATAPATH

Archive:  pneumonia.zip
  inflating: pneumonia-gpi-icd10-mimic3.parquet  
Archive:  notes.zip
  inflating: pneumonia-notes-mimic3.parquet  


In [6]:
pneumonia_df = pd.read_parquet('/content/pneumonia-gpi-icd10-mimic3.parquet', engine='pyarrow')
print(pneumonia_df.shape)
pneumonia_df.head()

(15159, 1683)


,HADM_ID,ETHNICITY,SUBJECT_ID,GENDER,ICD9_CODE,ICD9_SHORT_TITLE,ICD9_LONG_TITLE,PNEUMONIA,21351860002020,21100009102120,...,21500020102005.1,83100020302031,65991002052020,79700010002020,96897072002900,69100010102012,47100020100305,79993002202035,96424615652900,ICD10_CODE
0,128652,WHITE,31,M,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",1,0,0,...,0,0,0,0,0,0,0,0,0,J189
1,165660,WHITE,36,M,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",1,0,0,...,0,0,0,0,0,0,0,0,0,J189
2,188670,WHITE,37,M,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified",1,0,0,...,0,0,0,0,0,0,0,0,0,J189
3,101757,WHITE,41,M,48241,Meth sus pneum d/t Staph,Methicillin susceptible pneumonia due to Staph...,1,0,0,...,0,1,0,0,0,0,0,0,0,J15211
4,101757,WHITE,41,M,48282,Pneumonia e coli,Pneumonia due to escherichia coli [E. coli],1,0,0,...,0,1,0,0,0,0,0,0,0,J155


In [7]:
notes_df = pd.read_parquet('/content/pneumonia-notes-mimic3.parquet', engine='pyarrow')
print(notes_df.shape)
notes_df.head()

(553338, 11)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,175,13702,107527.0,2118-06-14,None,None,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
1,177,13702,196489.0,2124-08-18,None,None,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
2,178,26880,135453.0,2162-03-25,None,None,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
3,180,20646,134727.0,2112-12-10,None,None,Discharge summary,Report,NaN,NaN,Admission Date: [**2112-12-8**] ...
4,183,56174,189681.0,2118-12-09,None,None,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-12-7**] ...


In [9]:
NDC2GPI_URL = "https://drive.google.com/uc?export=download&id=1z61tRwAR0zBpFcA5oNWv1NpzEQEASHvb"
# https://drive.google.com/file/d/1z61tRwAR0zBpFcA5oNWv1NpzEQEASHvb/view?usp=sharing

NDC2GPI_DATAPATH = "ndc2gpi.json"
gdown.download(NDC2GPI_URL, NDC2GPI_DATAPATH, quiet=True)

'ndc2gpi.json'

In [12]:
with open(NDC2GPI_DATAPATH, 'r') as f:
    ndc2gpi_dict = json.load(f)
# ndc2gpi_dict

---

El problema de ML debe abordarse como una clasificación binaria (neumonia o no neumonia). Para ello, se recomienda aplicar CatBoost o XGBoost.

Se recomienda aplicar feature selección para reducir la cantidad de columnas ANTES de realizar la clasificación. Para ello puede utilizarse la biblioteca [feature-engine](https://feature-engine.trainindata.com/en/latest/)